In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client --upgrade google-api-python-client SpeechRecognition openai gtts ffmpeg-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.5/195.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.137.0
    Uninstalling google-api-python-client-2.137.0:
      Successfully uninsta

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving voicebot-431502-2f8677bedfec.json to voicebot-431502-2f8677bedfec.json


In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

def authenticate_google_calendar():
    SCOPES = ['https://www.googleapis.com/auth/calendar']
    SERVICE_ACCOUNT_FILE = '/content/voicebot-431502-2f8677bedfec.json'

    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('calendar', 'v3', credentials=credentials)
    return service

def create_event(summary, start_time, end_time):
    service = authenticate_google_calendar()

    event = {
        'summary': summary,
        'start': {
            'dateTime': start_time,
            'timeZone': 'America/New_York',
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'America/New_York',
        },
    }

    event_result = service.events().insert(calendarId='calender-id-here', body=event).execute()
    return f"Appointment scheduled: {event_result['summary']} from {event_result['start']['dateTime']} to {event_result['end']['dateTime']}"


summary = 'raviteja_testdrive_appointment'
start_time = '2024-08-02T19:00:00Z'
end_time = '2024-08-02T20:00:00Z'

result = create_event(summary, start_time, end_time)

print(result)

Appointment scheduled: raviteja_testdrive_appointment from 2024-08-02T15:00:00-04:00 to 2024-08-02T16:00:00-04:00


In [ ]:
from IPython.display import Javascript, Audio, display
from google.colab import output
from base64 import b64decode
import io
import subprocess
import speech_recognition as sr
from gtts import gTTS
from openai import OpenAI
import time
import os
from datetime import datetime, timedelta
from dateutil import parser
import re


RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = async () => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(10000)
  recorder.stop()
  return new Promise(resolve => {
    recorder.onstop = async () => {
      blob = new Blob(chunks)
      text = await b2text(blob)
      resolve(text)
    }
  })
}
"""

def record():
    display(Javascript(RECORD))
    s = output.eval_js('record()')
    b = b64decode(s.split(',')[1])
    filename = f"audio_{int(time.time())}.webm"
    with open(filename, "wb") as f:
        f.write(b)
    return filename

def convert_to_wav(input_file):
    output_file = f"{os.path.splitext(input_file)[0]}.wav"
    command = f"ffmpeg -i {input_file} -ac 1 -ar 16000 {output_file}"
    subprocess.run(command, shell=True)
    return output_file

def listen():
    audio_file = record()
    wav_file = convert_to_wav(audio_file)
    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_file) as source:
        audio_content = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio_content)
        #print(f"Recognized: {text}")
        return text
    except sr.UnknownValueError:
        print("Sorry, I did not understand that.")
        return None

def speak(text):
    print(f"Ai Assistant: {text}")
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    display(Audio("response.mp3", autoplay=True))
    audio_length = len(text.split()) / 2.5
    time.sleep(audio_length + 1)

showroom_data = {
    'location': '123 Main St, Buffalo, USA',
    'hours': 'Mon-Sat 9am-6pm, Sun 10am-4pm',
    'contact': '123-555-4567',
    'models': {
        'Model A': {
            'description': 'A luxury SUV with cutting-edge technology and performance.',
            'price': '$80,000',
            'features': ['Autopilot', 'All-Wheel Drive', 'Premium Interior']
        },
        'Model B': {
            'description': 'A high-performance electric sedan with long-range capabilities.',
            'price': '$90,000',
            'features': ['Autopilot', 'Long Range', 'Premium Sound System']
        },
        'Model C': {
            'description': 'An affordable electric car with great performance and safety features.',
            'price': '$40,000',
            'features': ['Autopilot', 'Rear-Wheel Drive', 'Full Self-Driving Capability']
        }
    }
}

def get_system_message():
    return f"""You are a car showroom assistant at XYZ Car Showroom.
    We offer a variety of new and used cars, including sedans, SUVs, and trucks.
    Our showroom is known for excellent customer service and competitive pricing.
    We also offer financing options and special discounts on certain models.

    Here is some additional information about our showroom:
    - Location: {showroom_data['location']}
    - Hours: {showroom_data['hours']}
    - Contact: {showroom_data['contact']}

    Our car models include:
    {', '.join([f"{model}: {data['description']}" for model, data in showroom_data['models'].items()])}

    Please provide information based on the following guidelines:
    - Only provide information about cars that are listed in the showroom_data.
    - Do not provide any information beyond what is included in showroom_data.
    - For appointments, follow the standard procedure as described.

    Assist customers by providing accurate details about our cars, comparing different models, and booking appointments for test drives or consultations."""

client = OpenAI(api_key="api-key-here")

def get_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": get_system_message()},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150
    )
    return response.choices[0].message.content.strip()

def get_car_details(model):
    if (details := showroom_data['models'].get(model)):
        return f"{model}: {details['description']} It is priced at {details['price']} and includes features like {', '.join(details['features'])}."
    else:
        return f"Sorry, I don't have details for {model}."

def handle_conversation():
    speak("Hello! This is your car showroom assistant. How can I assist you today?")
    while True:
        print("Speak Now...")
        user_input = listen()
        if user_input:
            print(f"User Input: {user_input}")

            if "appointment" in user_input.lower():
                speak("Sure, I can help you book an appointment. Please provide your name, preferred date and time.")
                print("Speak Now...")
                appointment_details = listen()
                if appointment_details:
                    print(f"Appointment Details: {appointment_details}")

                    gpt_prompt = (
                        f"Extract the following details from this input: '{appointment_details}'. "
                        f"Provide the details in the format: 'Name: [name], Date: [date], Time: [time]'."
                    )
                    extraction_result = get_response(gpt_prompt)
                    print(f"GPT Extraction Result: {extraction_result}")

                    try:
                        name_match = re.search(r'Name:\s*([^\s,]+(?:\s[^\s,]+)*)', extraction_result)
                        date_match = re.search(r'Date:\s*([^\s,]+(?:\s[^\s,]+)*)', extraction_result)
                        time_match = re.search(r'Time:\s*([^\s,]+(?:\s[^\s,]+)*)', extraction_result)

                        if not (name_match and date_match and time_match):
                            raise ValueError("Failed to extract all required details.")

                        name = name_match.group(1).strip()
                        date_str = date_match.group(1).strip()
                        time_str = time_match.group(1).strip()
                        date_time_str = f"{date_str} {time_str}"
                        date_time_obj = parser.parse(date_time_str)

                        start_time = date_time_obj.strftime('%Y-%m-%dT%H:%M:%S') + 'Z'
                        end_time = (date_time_obj + timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%S') + 'Z'

                        summary = f"Appointment with {name}"

                        result = create_event(summary, start_time, end_time)
                        speak(result)
                    except (ValueError, AttributeError) as e:
                        print(f"Error parsing extraction result: {e}")
                        speak("I could not understand the details provided. Please try again.")
                    #break

            elif "compare" in user_input.lower() or "comparison" in user_input.lower():
                speak("Sure, I can help you with a comparison. Please tell me the models you are interested in.")
                print("Speak Now...")
                models = listen()
                if models:
                    response = get_response(f"Compare the following car models: {models}")
                    speak(response)

            elif "details" in user_input.lower():
                speak("Sure, I can provide details. Please tell me the car model you are interested in.")
                print("Speak Now...")
                car_model = listen()
                if car_model:
                    response = get_car_details(car_model)
                    speak(response)

            else:
                response = get_response(user_input)
                speak(response)

        else:
            speak("I didn't catch that. Could you please repeat?")


handle_conversation()
